## Sisestused

Eelmises näites töötasime kõrgedimensionaalsete sõnakottide vektoritega, mille pikkus oli `vocab_size`, ja teisendasime madaladimensionaalsed positsiooniesituse vektorid selgesõnaliselt hõredaks ühekuumaks esitusviisiks. See ühekuum esitusviis ei ole mälusäästlik, lisaks käsitletakse iga sõna üksteisest sõltumatult, st ühekuum kodeeritud vektorid ei väljenda sõnade vahel semantilist sarnasust.

Selles osas jätkame **News AG** andmestiku uurimist. Alustuseks laadime andmed ja võtame mõned määratlused eelmisest märkmikust.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Mis on sisestamine?

**Sisestamise** idee seisneb selles, et sõnu esitatakse madalama dimensiooniga tihedate vektoritega, mis mingil moel kajastavad sõna semantilist tähendust. Hiljem arutame, kuidas luua tähenduslikke sõna sisestusi, kuid praegu mõtleme sisestustest lihtsalt kui viisist vähendada sõna vektori dimensioonilisust.

Sisestuskiht võtaks sõna sisendina ja annaks väljundvektori määratud `embedding_size` suurusega. Teatud mõttes on see väga sarnane `Linear` kihiga, kuid selle asemel, et võtta ühekuumkodeeritud vektor, suudab see võtta sisendiks sõna numbri.

Kasutades sisestuskihti meie võrgu esimesena kihina, saame liikuda sõnakoti mudelilt **sisestuskoti** mudelile, kus esmalt teisendame iga sõna tekstis vastavaks sisestuseks ja seejärel arvutame nende sisestuste üle mingi koondfunktsiooni, näiteks `sum`, `average` või `max`.

![Pilt, mis näitab viie sõna järjestuse sisestuse klassifikaatorit.](../../../../../translated_images/et/embedding-classifier-example.b77f021a7ee67eee.webp)

Meie klassifikaatori närvivõrk algab sisestuskihiga, millele järgneb koondamiskihiga ja lineaarne klassifikaator selle peal:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Töötamine muutuva järjestuse suurusega

Sellise arhitektuuri tulemusena tuleb meie võrgule minibatche luua kindlal viisil. Eelmises osas, kui kasutasime sõnakottide meetodit (bag-of-words), olid kõik BoW tensorid minibatchis ühesuurused, `vocab_size`, sõltumata meie tekstijärjestuse tegelikust pikkusest. Kui liigume sõnaembeddingsite kasutamisele, siis jõuame olukorda, kus igas tekstinäidises on erinev arv sõnu, ja nende näidiste minibatchidesse kombineerimisel peame rakendama täitmist (padding).

Seda saab teha, kasutades sama tehnikat, pakkudes andmeallikale `collate_fn` funktsiooni:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Embedding klassifikaatori treenimine

Nüüd, kui oleme määratlenud sobiva andmete laadija, saame mudelit treenida, kasutades treenimisfunktsiooni, mille määratlesime eelmises osas:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Märkus**: Treenime siin ainult 25 000 kirje jaoks (vähem kui üks täistsükkel) aja säästmiseks, kuid võite jätkata treenimist, kirjutada funktsiooni mitme tsükli treenimiseks ja katsetada õppemäära parameetriga, et saavutada suurem täpsus. Peaksite suutma saavutada umbes 90% täpsuse.


### EmbeddingBag kiht ja muutuva pikkusega järjestuste esitus

Eelmises arhitektuuris pidime kõik järjestused täitma sama pikkuseni, et need minibatch'i sobiksid. See ei ole kõige tõhusam viis muutuva pikkusega järjestuste esitamiseks - teine lähenemine oleks kasutada **nihke** vektorit, mis sisaldaks kõigi järjestuste nihked, mis on salvestatud ühes suures vektoris.

![Pilt, mis näitab nihkevektori järjestuse esitust](../../../../../translated_images/et/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Ülaloleval pildil on näidatud tähemärkide järjestus, kuid meie näites töötame sõnade järjestustega. Siiski jääb üldine põhimõte järjestuste esitamiseks nihkevektoriga samaks.

Nihkevektori esitusega töötamiseks kasutame [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) kihti. See on sarnane `Embedding`-iga, kuid võtab sisendiks sisuvektori ja nihkevektori ning sisaldab ka keskmistamise kihti, mis võib olla `mean`, `sum` või `max`.

Siin on muudetud võrk, mis kasutab `EmbeddingBag`-i:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Ettevalmistamaks andmekogumit treenimiseks, peame pakkuma teisendusfunktsiooni, mis valmistab ette nihkevektori:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Pange tähele, et erinevalt kõigist eelmistest näidetest võtab meie võrk nüüd vastu kaks parameetrit: andmevektori ja nihkevektori, mis on erineva suurusega. Samamoodi annab meie andmete laadija meile 3 väärtust 2 asemel: nii teksti- kui ka nihkevektorid antakse funktsioonidena. Seetõttu peame oma treeningfunktsiooni veidi kohandama, et sellega arvestada:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantilisedastused: Word2Vec

Eelmises näites õppis mudeli sisestuskiht kaardistama sõnu vektoriesitusteks, kuid sellel esitlusel puudus suurem semantiline tähendus. Oleks kasulik õppida sellist vektoriesitust, kus sarnased sõnad või sünonüümid vastaksid vektoritele, mis on teatud vektorkauguse (nt eukleidiline kaugus) mõttes üksteisele lähedal.

Selle saavutamiseks peame oma sisestusmudeli eelnevalt treenima suure tekstikogu peal kindlal viisil. Üks esimesi meetodeid semantiliste esinduste treenimiseks on [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). See põhineb kahel peamisel arhitektuuril, mida kasutatakse sõnade hajutatud esituse loomiseks:

 - **Järjepidev sõnakott** (CBoW) — selles arhitektuuris treenime mudelit ennustama sõna ümbritseva konteksti põhjal. Arvestades ngrammi $(W_{-2},W_{-1},W_0,W_1,W_2)$, on mudeli eesmärk ennustada $W_0$ $(W_{-2},W_{-1},W_1,W_2)$ põhjal.
 - **Järjepidev vahegramm** (Continuous skip-gram) on CBoW vastand. Mudel kasutab ümbritsevat kontekstiakent, et ennustada praegust sõna.

CBoW on kiirem, samas kui vahegramm on aeglasem, kuid esindab haruldasi sõnu paremini.

![Pilt, mis näitab nii CBoW kui ka Skip-Gram algoritme sõnade vektoriteks teisendamiseks.](../../../../../translated_images/et/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Et katsetada Word2Vec-i sisestust, mis on eelnevalt treenitud Google News andmekogul, saame kasutada **gensim** teeki. Allpool otsime sõnu, mis on kõige sarnasemad sõnale 'neural'.

> **Märkus:** Kui loote esimest korda sõnavektoreid, võib nende allalaadimine võtta aega!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Saame arvutada ka sõna vektorite sisestusi, mida kasutatakse klassifitseerimismudeli treenimisel (selguse huvides näitame ainult vektori esimesi 20 komponenti):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Suurepärane asi semantiliste sisestuste juures on see, et saate vektorkodeeringut manipuleerida, et muuta semantikat. Näiteks võime paluda leida sõna, mille vektorrepresentatsioon oleks võimalikult lähedal sõnadele *kuningas* ja *naine*, ning võimalikult kaugel sõnast *mees*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Nii CBoW kui ka Skip-Grams on "ennustavad" sõnaesitused, kuna need arvestavad ainult kohalikke kontekste. Word2Vec ei kasuta ära globaalset konteksti.

**FastText** põhineb Word2Vec-il, õppides vektoriesitusi iga sõna ja sõnas leiduvate tähemärkide n-grammide jaoks. Esituste väärtused keskmistatakse igal treeningusammul üheks vektoriks. Kuigi see lisab eeltreeningule palju lisaarvutusi, võimaldab see sõnaesitustel kodeerida alam-sõna teavet.

Teine meetod, **GloVe**, kasutab koosesinemise maatriksi ideed ja rakendab närvivõrkude meetodeid, et lagundada koosesinemise maatriks väljendusrikkamateks ja mittelineaarseteks sõnavektoriteks.

Saate näitega mängida, muutes sõnaesitusi FastTextiks ja GloVe'iks, kuna gensim toetab mitmeid erinevaid sõnaesitusmudeleid.


## Eelnevalt treenitud sisendvektorite kasutamine PyTorchis

Saame ülaltoodud näidet muuta nii, et täidame oma sisendvektori kihi maatriksi semantiliste sisendvektoritega, nagu Word2Vec. Tuleb arvestada, et eelnevalt treenitud sisendvektorite ja meie tekstikorpuse sõnavarad ei pruugi kokku langeda, mistõttu algväärtustame puuduvate sõnade kaalud juhuslike väärtustega:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Nüüd treenime oma mudelit. Pange tähele, et mudeli treenimiseks kuluv aeg on oluliselt pikem kui eelmises näites, kuna sisestuskihi suurus on suurem ja seega ka parameetrite arv palju suurem. Samuti, selle tõttu võib olla vajalik treenida mudelit rohkemate näidete peal, kui soovime vältida üleõppimist.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Meie puhul ei näe me suurt täpsuse kasvu, mis tõenäoliselt tuleneb üsna erinevatest sõnavaradest. 
Erinevate sõnavarade probleemi lahendamiseks saame kasutada ühte järgmistest lahendustest:
* Treenida word2vec mudel uuesti meie sõnavara põhjal
* Laadida meie andmestik eelnevalt treenitud word2vec mudeli sõnavaraga. Sõnavara, mida kasutatakse andmestiku laadimiseks, saab määrata laadimise ajal.

Viimane lähenemine tundub lihtsam, eriti kuna PyTorch `torchtext` raamistik sisaldab sisseehitatud tuge sisendvektoritele. Näiteks saame GloVe-põhise sõnavara luua järgmiselt:


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Laaditud sõnavaral on järgmised põhitoimingud:
* `vocab.stoi` sõnastik võimaldab meil muuta sõna selle sõnastiku indeksiks
* `vocab.itos` teeb vastupidist - muudab numbri sõnaks
* `vocab.vectors` on maatriks sisendvektoritest, seega sõna `s` sisendvektori saamiseks peame kasutama `vocab.vectors[vocab.stoi[s]]`

Siin on näide sisendvektoritega manipuleerimisest, et näidata võrrandit **kind-man+woman = queen** (pidin koefitsienti veidi kohandama, et see toimiks):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Klassifikaatori treenimiseks nende sisendite abil peame esmalt kodeerima oma andmekogu GloVe'i sõnavara abil:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Nagu me eespool nägime, salvestatakse kõik vektori manused `vocab.vectors` maatriksisse. See teeb nende kaalude laadimise manustamiskihi kaaludesse lihtsa kopeerimise abil väga lihtsaks:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Nüüd treenime oma mudelit ja vaatame, kas saame paremaid tulemusi:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Üks põhjusi, miks me ei näe märkimisväärset täpsuse suurenemist, on see, et mõned meie andmekogumi sõnad puuduvad eeltreenitud GloVe'i sõnavaras ja seetõttu neid sisuliselt ignoreeritakse. Selle probleemi ületamiseks saame treenida omaenda sõnaembeddingsid meie andmekogumi põhjal.


## Kontekstuaalsed sisendvektorid

Üks peamisi traditsiooniliste eeltreenitud sisendvektorite, nagu Word2Vec, piiranguid on sõna tähenduse eristamise probleem. Kuigi eeltreenitud sisendvektorid suudavad mingil määral tabada sõnade tähendust kontekstis, kodeeritakse iga sõna võimalikud tähendused samasse sisendvektorisse. See võib põhjustada probleeme järgnevatel mudelitel, kuna paljudel sõnadel, näiteks sõnal 'play', on erinevad tähendused sõltuvalt kontekstist, milles neid kasutatakse.

Näiteks sõnal 'play' on nendes kahes lauses üsna erinev tähendus:
- Ma käisin teatris **etendust** vaatamas.
- John tahab oma sõpradega **mängida**.

Eeltreenitud sisendvektorid esindavad mõlemat tähendust sõnast 'play' samas sisendvektoris. Selle piirangu ületamiseks peame looma sisendvektorid, mis põhinevad **keelemudelil**, mis on treenitud suurel tekstikorpusel ja *teab*, kuidas sõnu saab erinevates kontekstides kokku panna. Kontekstuaalsete sisendvektorite arutelu jääb selle õpetuse raamidest välja, kuid me tuleme nende juurde tagasi, kui räägime keelemudelitest järgmises osas.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
